### Імпортування даних у таблиці вимірів сховища даних

Імпортуємо бібліотеки

In [54]:
import psycopg2
import pandas as pd
from dotenv import dotenv_values

Завантажуємо змінні оточення з .env файлу для з'єднання зі сховищем

In [52]:
config = dotenv_values()

DB_NAME = config.get('DB_NAME')
DB_USER = config.get('DB_USER')
DB_HOST = config.get('DB_HOST')
DB_PASSWORD = config.get('DB_PASSWORD')
DB_PORT = config.get('DB_PORT')

З'єднуємося з базою даних

In [53]:
conn = psycopg2.connect(
    database=DB_NAME,
    user=DB_USER,
    host=DB_HOST,
    password=DB_PASSWORD,
    port=DB_PORT
)